# Quantum Process Tomography

* **Last Updated:** Jan 24, 2019
* **Requires:** qiskit-terra 0.7

In [1]:
# Needed for functions
import numpy as np
import time

# Import QISKit classes
import qiskit
from qiskit import QuantumRegister, QuantumCircuit, Aer
from qiskit.quantum_info import state_fidelity
from qiskit.tools.qi.qi import outer

# Tomography functions
import sys, os
sys.path.append(os.path.abspath(os.path.join('../qiskit_ignis')))
import tomography as tomo

## 1-qubit process tomography example

In [2]:
# Process tomography of a Hadamard gate
q = QuantumRegister(1)
circ = QuantumCircuit(q)
circ.h(q[0])

# Run circuit on unitary simulator to find ideal unitary
job = qiskit.execute(circ, Aer.get_backend('unitary_simulator'))
ideal_unitary = job.result().get_unitary(circ)
# convert to Choi-matrix in column-major convention
choi_ideal = outer(ideal_unitary.ravel(order='F'))

# Generate process tomography circuits and run on qasm simulator
qpt_circs = tomo.process_tomography_circuits(circ, q)
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=4000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_counts = tomo.tomography_data(job.result(), qpt_circs)
qpt_counts

{(('Zp',), ('X',)): {'0': 4000},
 (('Zp',), ('Y',)): {'1': 1960, '0': 2040},
 (('Zp',), ('Z',)): {'1': 1979, '0': 2021},
 (('Zm',), ('X',)): {'1': 4000},
 (('Zm',), ('Y',)): {'1': 2031, '0': 1969},
 (('Zm',), ('Z',)): {'1': 1978, '0': 2022},
 (('Xp',), ('X',)): {'1': 1993, '0': 2007},
 (('Xp',), ('Y',)): {'1': 2004, '0': 1996},
 (('Xp',), ('Z',)): {'0': 4000},
 (('Yp',), ('X',)): {'1': 2024, '0': 1976},
 (('Yp',), ('Y',)): {'1': 4000},
 (('Yp',), ('Z',)): {'1': 2098, '0': 1902}}

In [3]:
# Extract fitter data from tomography counts
data, basis, weights = tomo.fitter_data(qpt_counts)

# MLE Least-Squares tomographic reconstruction
t = time.time()
choi_mle = tomo.process_mle_fit(data, basis, weights)
print('MLE Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_mle / 2))

# CVXOPT Semidefinite-Program tomographic reconstruction
t = time.time()
choi_cvx = tomo.process_cvx_fit(data, basis, weights)
print('\nCVXOPT Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_cvx / 2))

MLE Fitter
fit time: 0.1457839012145996
fit fidelity: 0.9902245017121254

CVXOPT Fitter
fit time: 0.03708195686340332
fit fidelity: 0.9998921264767937


## 1-qubit process tomography of two-qubit swap gate

We will prepare qubit-0 and measure qubit-1 so the reconstructed channel should be an identity

In [4]:
# Process tomography of a Hadamard gate
q = QuantumRegister(2)
circ = QuantumCircuit(q)
circ.swap(q[0], q[1])

# Ideal channel is a unitary
ideal_unitary = np.eye(2)
choi_ideal = outer(ideal_unitary.ravel(order='F'))

# Generate process tomography circuits and run on qasm simulator
# We use the optional prepared_qubits kwarg to specify that the prepared qubit was different to measured qubit
qpt_circs = tomo.process_tomography_circuits(circ, q[1], prepared_qubits=q[0])
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=2000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_counts = tomo.tomography_data(job.result(), qpt_circs)
qpt_counts

{(('Zp',), ('X',)): {'1': 997, '0': 1003},
 (('Zp',), ('Y',)): {'1': 989, '0': 1011},
 (('Zp',), ('Z',)): {'0': 2000},
 (('Zm',), ('X',)): {'1': 1011, '0': 989},
 (('Zm',), ('Y',)): {'1': 1039, '0': 961},
 (('Zm',), ('Z',)): {'1': 2000},
 (('Xp',), ('X',)): {'0': 2000},
 (('Xp',), ('Y',)): {'1': 996, '0': 1004},
 (('Xp',), ('Z',)): {'1': 1019, '0': 981},
 (('Yp',), ('X',)): {'1': 1006, '0': 994},
 (('Yp',), ('Y',)): {'0': 2000},
 (('Yp',), ('Z',)): {'1': 972, '0': 1028}}

In [5]:
# Extract fitter data from tomography counts
data, basis, weights = tomo.fitter_data(qpt_counts)

# MLE Least-Squares tomographic reconstruction
t = time.time()
choi_mle = tomo.process_mle_fit(data, basis, weights)
print('MLE Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_mle / 2))

# CVXOPT Semidefinite-Program tomographic reconstruction
t = time.time()
choi_cvx = tomo.process_cvx_fit(data, basis, weights)
print('\nCVXOPT Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_cvx / 2))

MLE Fitter
fit time: 0.0014650821685791016
fit fidelity: 0.9956917452151154

CVXOPT Fitter
fit time: 0.028470993041992188
fit fidelity: 0.9999552921830998


## 2-Qubit entangling circuit

In [6]:
# Bell-state entangling circuit
q = QuantumRegister(2)
circ = QuantumCircuit(q)
circ.h(q[0])
circ.cx(q[0], q[1])

# Run circuit on unitary simulator to find ideal unitary
job = qiskit.execute(circ, Aer.get_backend('unitary_simulator'))
ideal_unitary = job.result().get_unitary(circ)
# convert to Choi-matrix in column-major convention
choi_ideal = outer(ideal_unitary.ravel(order='F'))

# Generate process tomography circuits and run on qasm simulator
qpt_circs = tomo.process_tomography_circuits(circ, q)
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=2000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_counts = tomo.tomography_data(job.result(), qpt_circs)

# Extract fitter data from tomography counts
data, basis, weights = tomo.fitter_data(qpt_counts)


t = time.time()
choi_mle = tomo.process_mle_fit(data, basis, weights)
print('MLE Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 4, choi_mle / 4))

t = time.time()
choi_cvx = tomo.process_cvx_fit(data, basis, weights)
print('\nCVXOPT Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 4, choi_cvx / 4))

MLE Fitter
fit time: 0.04158282279968262
fit fidelity: 0.9834101265402414

CVXOPT Fitter
fit time: 1.961974859237671
fit fidelity: 0.9999306422410397


## Using SIC-POVM preparation basis

In [7]:
# Process tomography of a Hadamard gate
q = QuantumRegister(1)
circ = QuantumCircuit(q)
circ.h(q[0])

# Run circuit on unitary simulator to find ideal unitary
job = qiskit.execute(circ, Aer.get_backend('unitary_simulator'))
ideal_unitary = job.result().get_unitary(circ)
# convert to Choi-matrix in column-major convention
choi_ideal = outer(ideal_unitary.ravel(order='F'))

# Generate process tomography circuits and run on qasm simulator
qpt_circs = tomo.process_tomography_circuits(circ, q, prep_labels='SIC', prep_basis='SIC')
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=2000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_counts = tomo.tomography_data(job.result(), qpt_circs)
qpt_counts

{(('S0',), ('X',)): {'0': 2000},
 (('S0',), ('Y',)): {'1': 1010, '0': 990},
 (('S0',), ('Z',)): {'1': 989, '0': 1011},
 (('S1',), ('X',)): {'1': 1345, '0': 655},
 (('S1',), ('Y',)): {'1': 965, '0': 1035},
 (('S1',), ('Z',)): {'1': 60, '0': 1940},
 (('S2',), ('X',)): {'1': 1314, '0': 686},
 (('S2',), ('Y',)): {'1': 191, '0': 1809},
 (('S2',), ('Z',)): {'1': 1468, '0': 532},
 (('S3',), ('X',)): {'1': 1345, '0': 655},
 (('S3',), ('Y',)): {'1': 1826, '0': 174},
 (('S3',), ('Z',)): {'1': 1419, '0': 581}}

In [8]:
# Extract fitter data from tomography counts
data, basis, weights = tomo.fitter_data(qpt_counts, prep_basis='SIC')

# MLE Least-Squares tomographic reconstruction
t = time.time()
choi_mle = tomo.process_mle_fit(data, basis, weights)
print('MLE Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_mle / 2))

# CVXOPT Semidefinite-Program tomographic reconstruction
t = time.time()
choi_cvx = tomo.process_cvx_fit(data, basis, weights)

print('\nCVXOPT Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_cvx / 2))

MLE Fitter
fit time: 0.002043008804321289
fit fidelity: 0.9923398831786698

CVXOPT Fitter
fit time: 0.07675480842590332
fit fidelity: 0.9938897085080838
